# Regression Analysis
## Preliminary Set-Up
This do-file analyses the impact of Hurricanes Katrina and Sandy on the migration systems of the affected area by using first a modified gravity model, that compares variation in inflows to 
disaster affected and nearby counties after the Hurricanes, and then a difference-in-differences model with fixed-effects for sending-receiving county pairs, to understand whether there is evidence of recovery migration when comparing the nearby and the disaster-affected counties.

In [ ]:
set more off, permanently
set matsize 11000

Here you should add the directory where the ```gravity_data.csv``` file produced by the ```regression_analysis``` notebook is located.

In [ ]:
cd "your/directory"

We import the dataset

In [ ]:
clear all
import delimited "gravity_data.csv"

We generate a numeric version of the year variable. The year string variable containes pairs of years like year1-year2, the numeric variable extracts only year1, which is the year to which most of the migration flows refer to.

In [ ]:
generate year_num = substr(year,1,4)
destring year_num, replace

We generate dummy variables for time (before and after the Hurricane) and treatment (being in disaster affected counties for the Hurricane) for each Hurricane.

In [ ]:
generate time_sandy = 0
replace time_sandy = 1 if inlist(year, "2012-2013", "2013-2014")

generate time_kat = 0
replace time_kat = 1 if inlist(year, "2005-2006", "2006-2007", "2007-2008")

generate treatment_sandy = 0
replace treatment_sandy = 1 if sandy_group_dest == "disaster"

generate treatment_kat = 0
replace treatment_kat = 1 if kat_group_dest == "disaster"

We also generate the time*treatment interaction to denote the flows which refer to disaster-affected counties after the Hurricane. This is our treatment group.

In [ ]:
generate treated_kat = 0
replace treated_kat = 1 if (treatment_kat == 1 & time_kat == 1)

generate treated_sandy = 0
replace treated_sandy = 1 if (treatment_sandy == 1 & time_sandy == 1)

We generate a dummy for each sending-receiving county pair

In [ ]:
egen sending_receiving = group(destination origin)

We apply a log transformation to the following variables:

1. ```return_num```: which is an estimate of the number of households that moved from one county to the other;
2. ```exmpt_num```: which is an estimate of the number of individuals that moved from one county to the other;
3. ```pop_destination_hh```: which is an estimate of the number of households at the destination;
4. ```pop_origin_hh```: which is an estimate of the number of households at the origin;
5. ```pop_destination_in```: which is an estimate of the number of individuals at the destination;
6. ```pop_origin_in```: which is an estimate of the number of individuals at the origin;
				

In [ ]:
replace return_num = log(return_num)
replace exmpt_num = log(exmpt_num)
replace pop_destination_hh = log(pop_destination_hh)
replace pop_origin_hh = log(pop_origin_hh)
replace pop_destination_in = log(pop_destination_in)
replace pop_origin_in = log(pop_origin_in)

We sort the dataset and remove duplicates

In [ ]:
sort sending_receiving year_num
quietly by sending_receiving year_num:  gen dup = cond(_N==1,0,_n)
drop if dup>1

We set ```sending_receiving``` as the panel variable and ```year_num``` as the time variable

In [ ]:
xtset sending_receiving year_num

We can now proceed with the analysis. First I will perform the analsysis for Katrina, then the one for Sandy. The regression results are written to a ```.csv``` file.

## Katrina Replication Analysis
### Gravity Model
#### Disaster-All

In [ ]:
eststo: xtreg exmpt_num pop_origin_in pop_destination_in i.time_kat ///
			if ((inlist(year, "1999-2000", "2000-2001", "2001-2002", ///
							  "2002-2003", "2003-2004", "2004-2005", ///
							  "2007-2008", "2008-2009", "2009-2010")) & ///
			(kat_group_dest == "disaster")), ///
			vce(robust) fe

#### Disaster-Disaster

In [ ]:
eststo: xtreg exmpt_num pop_origin_in pop_destination_in i.time_kat ///
			if ((inlist(year, "1999-2000", "2000-2001", "2001-2002", ///
							  "2002-2003", "2003-2004", "2004-2005", ///
							  "2007-2008", "2008-2009", "2009-2010")) & ///
			(inlist(kat_group_dest, "disaster")) & ///
			(kat_group_origin == "disaster")), ///
			vce(robust) fe

#### Disaster-Nearby

In [ ]:
eststo: xtreg exmpt_num pop_origin_in pop_destination_in i.time_kat ///
			if ((inlist(year, "1999-2000", "2000-2001", "2001-2002", ///
							  "2002-2003", "2003-2004", "2004-2005", ///
							  "2007-2008", "2008-2009", "2009-2010")) & ///
			(inlist(kat_group_dest, "disaster")) & ///
			(kat_group_origin == "nearby")), ///
			vce(robust) fe

#### Disaster-Distant

In [ ]:
eststo: xtreg exmpt_num pop_origin_in pop_destination_in i.time_kat ///
			if ((inlist(year, "1999-2000", "2000-2001", "2001-2002", ///
							  "2002-2003", "2003-2004", "2004-2005", ///
							  "2007-2008", "2008-2009", "2009-2010")) & ///
			(inlist(kat_group_dest, "disaster")) & ///
			(kat_group_origin == "distant")), ///
			vce(robust) fe

#### Disaster-All (Urban)

In [ ]:
eststo: xtreg exmpt_num pop_origin_in pop_destination_in i.time_kat ///
			if ((inlist(year, "1999-2000", "2000-2001", "2001-2002", ///
							  "2002-2003", "2003-2004", "2004-2005", ///
							  "2007-2008", "2008-2009", "2009-2010")) & ///
			(inlist(kat_group_dest, "disaster")) & ///
			(urban_origin == "urban")), ///
			vce(robust) fe

#### Disaster-Disaster (Urban)

In [ ]:
eststo: xtreg exmpt_num pop_origin_in pop_destination_in i.time_kat ///
			if ((inlist(year, "1999-2000", "2000-2001", "2001-2002", ///
							  "2002-2003", "2003-2004", "2004-2005", ///
							  "2007-2008", "2008-2009", "2009-2010")) & ///
			(inlist(kat_group_dest, "disaster")) & ///
			(urban_origin == "urban") & ///
			(kat_group_origin == "disaster")), ///
			vce(robust) fe

#### Disaster-Nearby (Urban)

In [ ]:
eststo: xtreg exmpt_num pop_origin_in pop_destination_in i.time_kat ///
			if ((inlist(year, "1999-2000", "2000-2001", "2001-2002", ///
							  "2002-2003", "2003-2004", "2004-2005", ///
							  "2007-2008", "2008-2009", "2009-2010")) & ///
			(inlist(kat_group_dest, "disaster")) & ///
			(urban_origin == "urban") & ///
			(kat_group_origin == "nearby")), ///
			vce(robust) fe

#### Disaster-Distant (Urban)

In [ ]:
eststo: xtreg exmpt_num pop_origin_in pop_destination_in i.time_kat ///
			if ((inlist(year, "1999-2000", "2000-2001", "2001-2002", ///
							  "2002-2003", "2003-2004", "2004-2005", ///
							  "2007-2008", "2008-2009", "2009-2010")) & ///
			(inlist(kat_group_dest, "disaster")) & ///
			(urban_origin == "urban") & ///
			(kat_group_origin == "distant")), ///
			vce(robust) fe

#### Storing the estimates

In [ ]:
esttab using "gravity_disaster_kat.csv", ///
		replace ar2 csv noomitted nolines label se
	
eststo clear

#### Nearby-All

In [ ]:
eststo: xtreg exmpt_num pop_origin_in pop_destination_in i.time_kat ///
			if ((inlist(year, "1999-2000", "2000-2001", "2001-2002", ///
							  "2002-2003", "2003-2004", "2004-2005", ///
							  "2007-2008", "2008-2009", "2009-2010")) & ///
			(kat_group_dest == "nearby")), ///
			vce(robust) fe

#### Nearby-Disaster

In [ ]:
eststo: xtreg exmpt_num pop_origin_in pop_destination_in i.time_kat ///
			if ((inlist(year, "1999-2000", "2000-2001", "2001-2002", ///
							  "2002-2003", "2003-2004", "2004-2005", ///
							  "2007-2008", "2008-2009", "2009-2010")) & ///
			(inlist(kat_group_dest, "nearby")) & ///
			(kat_group_origin == "disaster")), ///
			vce(robust) fe

#### Nearby-Nearby

In [ ]:
eststo: xtreg exmpt_num pop_origin_in pop_destination_in i.time_kat ///
			if ((inlist(year, "1999-2000", "2000-2001", "2001-2002", ///
							  "2002-2003", "2003-2004", "2004-2005", ///
							  "2007-2008", "2008-2009", "2009-2010")) & ///
			(inlist(kat_group_dest, "nearby")) & ///
			(kat_group_origin == "nearby")), ///
			vce(robust) fe

#### Nearby-Distant

In [ ]:
eststo: xtreg exmpt_num pop_origin_in pop_destination_in i.time_kat ///
			if ((inlist(year, "1999-2000", "2000-2001", "2001-2002", ///
							  "2002-2003", "2003-2004", "2004-2005", ///
							  "2007-2008", "2008-2009", "2009-2010")) & ///
			(inlist(kat_group_dest, "nearby")) & ///
			(kat_group_origin == "distant")), ///
			vce(robust) fe

#### Nearby-All (Urban)

In [ ]:
eststo: xtreg exmpt_num pop_origin_in pop_destination_in i.time_kat ///
			if ((inlist(year, "1999-2000", "2000-2001", "2001-2002", ///
							  "2002-2003", "2003-2004", "2004-2005", ///
							  "2007-2008", "2008-2009", "2009-2010")) & ///
			(inlist(kat_group_dest, "nearby")) & ///
			(urban_origin == "urban")), ///
			vce(robust) fe

#### Nearby-Disaster (Urban)

In [ ]:
eststo: xtreg exmpt_num pop_origin_in pop_destination_in i.time_kat ///
			if ((inlist(year, "1999-2000", "2000-2001", "2001-2002", ///
							  "2002-2003", "2003-2004", "2004-2005", ///
							  "2007-2008", "2008-2009", "2009-2010")) & ///
			(inlist(kat_group_dest, "nearby")) & ///
			(urban_origin == "urban") & ///
			(kat_group_origin == "disaster")), ///
			vce(robust) fe

#### Nearby-Nearby (Urban)

In [ ]:
eststo: xtreg exmpt_num pop_origin_in pop_destination_in i.time_kat ///
			if ((inlist(year, "1999-2000", "2000-2001", "2001-2002", ///
							  "2002-2003", "2003-2004", "2004-2005", ///
							  "2007-2008", "2008-2009", "2009-2010")) & ///
			(inlist(kat_group_dest, "nearby")) & ///
			(urban_origin == "urban") & ///
			(kat_group_origin == "nearby")), ///
			vce(robust) fe

#### Nearby-Distant (Urban)

In [ ]:
eststo: xtreg exmpt_num pop_origin_in pop_destination_in i.time_kat ///
			if ((inlist(year, "1999-2000", "2000-2001", "2001-2002", ///
							  "2002-2003", "2003-2004", "2004-2005", ///
							  "2007-2008", "2008-2009", "2009-2010")) & ///
			(inlist(kat_group_dest, "nearby")) & ///
			(urban_origin == "urban") & ///
			(kat_group_origin == "distant")), ///
			vce(robust) fe

#### Storing the estimates

In [ ]:
esttab using "gravity_nearby_kat.csv", ///
		replace ar2 csv noomitted nolines label se
	
eststo clear

### Difference-in-Differences Model
#### All

In [ ]:
eststo: xtreg exmpt_num pop_origin_in pop_destination_in i.time_kat i.treated_kat ///
		   if (inlist(year, "1999-2000", "2000-2001", "2001-2002", ///
							"2002-2003", "2003-2004", "2004-2005", ///
							"2005-2006", "2006-2007", "2007-2008") & ///
		   inlist(kat_group_dest, "disaster", "nearby")), ///
		   vce(robust) fe

#### Disaster

In [ ]:
eststo: xtreg exmpt_num pop_origin_in pop_destination_in i.time_kat i.treated_kat ///
		   if (inlist(year, "1999-2000", "2000-2001", "2001-2002", ///
							"2002-2003", "2003-2004", "2004-2005", ///
							"2005-2006", "2006-2007", "2007-2008") & ///
		   inlist(kat_group_dest, "disaster", "nearby") & ///
		   (kat_group_origin == "disaster")), ///
		   vce(robust) fe

#### Nearby

In [ ]:
eststo: xtreg exmpt_num pop_origin_in pop_destination_in i.time_kat i.treated_kat ///
		   if (inlist(year, "1999-2000", "2000-2001", "2001-2002", ///
							"2002-2003", "2003-2004", "2004-2005", ///
							"2005-2006", "2006-2007", "2007-2008") & ///
		   inlist(kat_group_dest, "disaster", "nearby") & ///
		   (urban_origin == "urban") & ///
		   (kat_group_origin == "nearby")), ///
		   vce(robust) fe

#### Distant

In [ ]:
eststo: xtreg exmpt_num pop_origin_in pop_destination_in i.time_kat i.treated_kat ///
		   if (inlist(year, "1999-2000", "2000-2001", "2001-2002", ///
							"2002-2003", "2003-2004", "2004-2005", ///
							"2005-2006", "2006-2007", "2007-2008") & ///
		   inlist(kat_group_dest, "disaster", "nearby") & ///
		   (urban_origin == "urban") & ///
		   (kat_group_origin == "distant")), ///
		   vce(robust) fe

#### All (Urban)

In [ ]:
eststo: xtreg exmpt_num pop_origin_in pop_destination_in i.time_kat i.treated_kat ///
		   if (inlist(year, "1999-2000", "2000-2001", "2001-2002", ///
							"2002-2003", "2003-2004", "2004-2005", ///
							"2005-2006", "2006-2007", "2007-2008") & ///
		   inlist(kat_group_dest, "disaster", "nearby") & ///
		   (urban_origin == "urban")), ///
		   vce(robust) fe

#### Disaster (Urban)

In [ ]:
eststo: xtreg exmpt_num pop_origin_in pop_destination_in i.time_kat i.treated_kat ///
		   if (inlist(year, "1999-2000", "2000-2001", "2001-2002", ///
							"2002-2003", "2003-2004", "2004-2005", ///
							"2005-2006", "2006-2007", "2007-2008") & ///
		   inlist(kat_group_dest, "disaster", "nearby") & ///
		   urban_origin == "urban" & ///
		   (kat_group_origin == "disaster")), ///
		   vce(robust) fe

#### Nearby (Urban)

In [ ]:
eststo: xtreg exmpt_num pop_origin_in pop_destination_in i.time_kat i.treated_kat ///
			if (inlist(year, "1999-2000", "2000-2001", "2001-2002", ///
							 "2002-2003", "2003-2004", "2004-2005", ///
							 "2005-2006", "2006-2007", "2007-2008") & ///
		   inlist(kat_group_dest, "disaster", "nearby") & ///
		   urban_origin == "urban" & ///
		   (kat_group_origin == "nearby")), ///
		   vce(robust) fe

#### Distant (Urban)

In [ ]:
eststo: xtreg exmpt_num pop_origin_in pop_destination_in i.time_kat i.treated_kat ///
			if (inlist(year, "1999-2000", "2000-2001", "2001-2002", ///
							 "2002-2003", "2003-2004", "2004-2005", ///
							 "2005-2006", "2006-2007", "2007-2008") & ///
		   inlist(kat_group_dest, "disaster", "nearby") & ///
		   urban_origin == "urban" & ///
		   (kat_group_origin == "distant")), ///
		   vce(robust) fe

#### Storing the Estimates

In [ ]:
esttab using "diffindiff_results_kat.csv", ///
	replace ar2 csv noomitted nolines label se

eststo clear	

## Sandy Replication Analysis
### Gravity Model
#### Disaster-All

In [ ]:
eststo: xtreg exmpt_num pop_origin_in pop_destination_in i.time_sandy ///
			if ((inlist(year, "1999-2000", "2000-2001", "2001-2002", ///
							  "2002-2003", "2003-2004", "2004-2005", ///
							  "2007-2008", "2008-2009", "2009-2010")) & ///
			(sandy_group_dest == "disaster")), ///
			vce(robust) fe

#### Disaster-Disaster

In [ ]:
eststo: xtreg exmpt_num pop_origin_in pop_destination_in i.time_sandy ///
			if ((inlist(year, "1999-2000", "2000-2001", "2001-2002", ///
							  "2002-2003", "2003-2004", "2004-2005", ///
							  "2007-2008", "2008-2009", "2009-2010")) & ///
			(inlist(sandy_group_dest, "disaster")) & ///
			(sandy_group_origin == "disaster")), ///
			vce(robust) fe

#### Disaster-Nearby

In [ ]:
eststo: xtreg exmpt_num pop_origin_in pop_destination_in i.time_sandy ///
			if ((inlist(year, "1999-2000", "2000-2001", "2001-2002", ///
							  "2002-2003", "2003-2004", "2004-2005", ///
							  "2007-2008", "2008-2009", "2009-2010")) & ///
			(inlist(sandy_group_dest, "disaster")) & ///
			(sandy_group_origin == "nearby")), ///
			vce(robust) fe

#### Disaster-Distant

In [ ]:
eststo: xtreg exmpt_num pop_origin_in pop_destination_in i.time_sandy ///
			if ((inlist(year, "1999-2000", "2000-2001", "2001-2002", ///
							  "2002-2003", "2003-2004", "2004-2005", ///
							  "2007-2008", "2008-2009", "2009-2010")) & ///
			(inlist(sandy_group_dest, "disaster")) & ///
			(sandy_group_origin == "distant")), ///
			vce(robust) fe

#### Disaster-All (Urban)

In [ ]:
eststo: xtreg exmpt_num pop_origin_in pop_destination_in i.time_sandy ///
			if ((inlist(year, "1999-2000", "2000-2001", "2001-2002", ///
							  "2002-2003", "2003-2004", "2004-2005", ///
							  "2007-2008", "2008-2009", "2009-2010")) & ///
			(inlist(sandy_group_dest, "disaster")) & ///
			(urban_origin == "urban")), ///
			vce(robust) fe

#### Disaster-Disaster (Urban)

In [ ]:
eststo: xtreg exmpt_num pop_origin_in pop_destination_in i.time_sandy ///
			if ((inlist(year, "1999-2000", "2000-2001", "2001-2002", ///
							  "2002-2003", "2003-2004", "2004-2005", ///
							  "2007-2008", "2008-2009", "2009-2010")) & ///
			(inlist(sandy_group_dest, "disaster")) & ///
			(urban_origin == "urban") & ///
			(sandy_group_origin == "disaster")), ///
			vce(robust) fe

#### Disaster-Nearby (Urban)

In [ ]:
eststo: xtreg exmpt_num pop_origin_in pop_destination_in i.time_sandy ///
			if ((inlist(year, "1999-2000", "2000-2001", "2001-2002", ///
							  "2002-2003", "2003-2004", "2004-2005", ///
							  "2007-2008", "2008-2009", "2009-2010")) & ///
			(inlist(sandy_group_dest, "disaster")) & ///
			(urban_origin == "urban") & ///
			(sandy_group_origin == "nearby")), ///
			vce(robust) fe

#### Disaster-Distant (Urban)

In [ ]:
eststo: xtreg exmpt_num pop_origin_in pop_destination_in i.time_sandy ///
			if ((inlist(year, "1999-2000", "2000-2001", "2001-2002", ///
							  "2002-2003", "2003-2004", "2004-2005", ///
							  "2007-2008", "2008-2009", "2009-2010")) & ///
			(inlist(sandy_group_dest, "disaster")) & ///
			(urban_origin == "urban") & ///
			(sandy_group_origin == "distant")), ///
			vce(robust) fe

#### Storing the estimates

In [ ]:
esttab using "gravity_disaster_sandy.csv", ///
		replace ar2 csv noomitted nolines label se
	
eststo clear

#### Nearby-All

In [ ]:
eststo: xtreg exmpt_num pop_origin_in pop_destination_in i.time_sandy ///
			if ((inlist(year, "1999-2000", "2000-2001", "2001-2002", ///
							  "2002-2003", "2003-2004", "2004-2005", ///
							  "2007-2008", "2008-2009", "2009-2010")) & ///
			(sandy_group_dest == "nearby")), ///
			vce(robust) fe

#### Nearby-Disaster

In [ ]:
eststo: xtreg exmpt_num pop_origin_in pop_destination_in i.time_sandy ///
			if ((inlist(year, "1999-2000", "2000-2001", "2001-2002", ///
							  "2002-2003", "2003-2004", "2004-2005", ///
							  "2007-2008", "2008-2009", "2009-2010")) & ///
			(inlist(sandy_group_dest, "nearby")) & ///
			(sandy_group_origin == "disaster")), ///
			vce(robust) fe

#### Nearby-Nearby

In [ ]:
eststo: xtreg exmpt_num pop_origin_in pop_destination_in i.time_sandy ///
			if ((inlist(year, "1999-2000", "2000-2001", "2001-2002", ///
							  "2002-2003", "2003-2004", "2004-2005", ///
							  "2007-2008", "2008-2009", "2009-2010")) & ///
			(inlist(sandy_group_dest, "nearby")) & ///
			(sandy_group_origin == "nearby")), ///
			vce(robust) fe

#### Nearby-Distant

In [ ]:
eststo: xtreg exmpt_num pop_origin_in pop_destination_in i.time_sandy ///
			if ((inlist(year, "1999-2000", "2000-2001", "2001-2002", ///
							  "2002-2003", "2003-2004", "2004-2005", ///
							  "2007-2008", "2008-2009", "2009-2010")) & ///
			(inlist(sandy_group_dest, "nearby")) & ///
			(sandy_group_origin == "distant")), ///
			vce(robust) fe

#### Nearby-All (Urban)

In [ ]:
eststo: xtreg exmpt_num pop_origin_in pop_destination_in i.time_sandy ///
			if ((inlist(year, "1999-2000", "2000-2001", "2001-2002", ///
							  "2002-2003", "2003-2004", "2004-2005", ///
							  "2007-2008", "2008-2009", "2009-2010")) & ///
			(inlist(sandy_group_dest, "nearby")) & ///
			(urban_origin == "urban")), ///
			vce(robust) fe

#### Nearby-Disaster (Urban)

In [ ]:
eststo: xtreg exmpt_num pop_origin_in pop_destination_in i.time_sandy ///
			if ((inlist(year, "1999-2000", "2000-2001", "2001-2002", ///
							  "2002-2003", "2003-2004", "2004-2005", ///
							  "2007-2008", "2008-2009", "2009-2010")) & ///
			(inlist(sandy_group_dest, "nearby")) & ///
			(urban_origin == "urban") & ///
			(sandy_group_origin == "disaster")), ///
			vce(robust) fe

#### Nearby-Nearby (Urban)

In [ ]:
eststo: xtreg exmpt_num pop_origin_in pop_destination_in i.time_sandy ///
			if ((inlist(year, "1999-2000", "2000-2001", "2001-2002", ///
							  "2002-2003", "2003-2004", "2004-2005", ///
							  "2007-2008", "2008-2009", "2009-2010")) & ///
			(inlist(sandy_group_dest, "nearby")) & ///
			(urban_origin == "urban") & ///
			(sandy_group_origin == "nearby")), ///
			vce(robust) fe

#### Nearby-Distant (Urban)

In [ ]:
eststo: xtreg exmpt_num pop_origin_in pop_destination_in i.time_sandy ///
			if ((inlist(year, "1999-2000", "2000-2001", "2001-2002", ///
							  "2002-2003", "2003-2004", "2004-2005", ///
							  "2007-2008", "2008-2009", "2009-2010")) & ///
			(inlist(sandy_group_dest, "nearby")) & ///
			(urban_origin == "urban") & ///
			(sandy_group_origin == "distant")), ///
			vce(robust) fe

#### Storing the estimates

In [ ]:
esttab using "gravity_nearby_sandy.csv", ///
		replace ar2 csv noomitted nolines label se
	
eststo clear

### Difference-in-Differences Model
#### All

In [ ]:
eststo: xtreg exmpt_num pop_origin_in pop_destination_in i.time_sandy i.treated_sandy ///
		   if (inlist(year, "1999-2000", "2000-2001", "2001-2002", ///
							"2002-2003", "2003-2004", "2004-2005", ///
							"2005-2006", "2006-2007", "2007-2008") & ///
		   inlist(sandy_group_dest, "disaster", "nearby")), ///
		   vce(robust) fe

#### Disaster

In [ ]:
eststo: xtreg exmpt_num pop_origin_in pop_destination_in i.time_sandy i.treated_sandy ///
		   if (inlist(year, "1999-2000", "2000-2001", "2001-2002", ///
							"2002-2003", "2003-2004", "2004-2005", ///
							"2005-2006", "2006-2007", "2007-2008") & ///
		   inlist(sandy_group_dest, "disaster", "nearby") & ///
		   (sandy_group_origin == "disaster")), ///
		   vce(robust) fe

#### Nearby

In [ ]:
eststo: xtreg exmpt_num pop_origin_in pop_destination_in i.time_sandy i.treated_sandy ///
		   if (inlist(year, "1999-2000", "2000-2001", "2001-2002", ///
							"2002-2003", "2003-2004", "2004-2005", ///
							"2005-2006", "2006-2007", "2007-2008") & ///
		   inlist(sandy_group_dest, "disaster", "nearby") & ///
		   (urban_origin == "urban") & ///
		   (sandy_group_origin == "nearby")), ///
		   vce(robust) fe

#### Distant

In [ ]:
eststo: xtreg exmpt_num pop_origin_in pop_destination_in i.time_sandy i.treated_sandy ///
		   if (inlist(year, "1999-2000", "2000-2001", "2001-2002", ///
							"2002-2003", "2003-2004", "2004-2005", ///
							"2005-2006", "2006-2007", "2007-2008") & ///
		   inlist(sandy_group_dest, "disaster", "nearby") & ///
		   (urban_origin == "urban") & ///
		   (sandy_group_origin == "distant")), ///
		   vce(robust) fe

#### All (Urban)

In [ ]:
eststo: xtreg exmpt_num pop_origin_in pop_destination_in i.time_sandy i.treated_sandy ///
		   if (inlist(year, "1999-2000", "2000-2001", "2001-2002", ///
							"2002-2003", "2003-2004", "2004-2005", ///
							"2005-2006", "2006-2007", "2007-2008") & ///
		   inlist(sandy_group_dest, "disaster", "nearby") & ///
		   (urban_origin == "urban")), ///
		   vce(robust) fe

#### Disaster (Urban)

In [ ]:
eststo: xtreg exmpt_num pop_origin_in pop_destination_in i.time_sandy i.treated_sandy ///
		   if (inlist(year, "1999-2000", "2000-2001", "2001-2002", ///
							"2002-2003", "2003-2004", "2004-2005", ///
							"2005-2006", "2006-2007", "2007-2008") & ///
		   inlist(sandy_group_dest, "disaster", "nearby") & ///
		   urban_origin == "urban" & ///
		   (sandy_group_origin == "disaster")), ///
		   vce(robust) fe

#### Nearby (Urban)

In [ ]:
eststo: xtreg exmpt_num pop_origin_in pop_destination_in i.time_sandy i.treated_sandy ///
			if (inlist(year, "1999-2000", "2000-2001", "2001-2002", ///
							 "2002-2003", "2003-2004", "2004-2005", ///
							 "2005-2006", "2006-2007", "2007-2008") & ///
		   inlist(sandy_group_dest, "disaster", "nearby") & ///
		   urban_origin == "urban" & ///
		   (sandy_group_origin == "nearby")), ///
		   vce(robust) fe

#### Distant (Urban)

In [ ]:
eststo: xtreg exmpt_num pop_origin_in pop_destination_in i.time_sandy i.treated_sandy ///
			if (inlist(year, "1999-2000", "2000-2001", "2001-2002", ///
							 "2002-2003", "2003-2004", "2004-2005", ///
							 "2005-2006", "2006-2007", "2007-2008") & ///
		   inlist(sandy_group_dest, "disaster", "nearby") & ///
		   urban_origin == "urban" & ///
		   (sandy_group_origin == "distant")), ///
		   vce(robust) fe

#### Storing the Estimates

In [ ]:
esttab using "diffindiff_results_sandy.csv", ///
	replace ar2 csv noomitted nolines label se

eststo clear	